In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bengaliai-cv19/test_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test_image_data_3.parquet
/kaggle/input/bengaliai-cv19/test_image_data_0.parquet
/kaggle/input/bengaliai-cv19/train.csv
/kaggle/input/bengaliai-cv19/class_map_corrected.csv
/kaggle/input/bengaliai-cv19/test_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_multi_diacritics.csv
/kaggle/input/bengaliai-cv19/class_map.csv
/kaggle/input/bengaliai-cv19/train_image_data_3.parquet
/kaggle/input/bengaliai-cv19/train_image_data_2.parquet
/kaggle/input/bengaliai-cv19/test.csv
/kaggle/input/bengaliai-cv19/sample_submission.csv
/kaggle/input/bengaliai-cv19/train_image_data_1.parquet
/kaggle/input/bengaliai-cv19/train_image_data_0.parquet
/kaggle/input/resnet5091acc/acc91_resnet50_saved_weights_1585840262.pth


In [2]:
# PACKAGES
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import os
import pdb
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from torchvision.models import resnet50,resnet18
from torch import nn
import torch.nn.functional as F
from torch.nn import Sequential
import time 
import copy
import cv2
import imgaug
import gc

In [3]:
import imgaug.augmenters as iaa
import albumentations

In [4]:
res = albumentations.augmentations.transforms.Resize(256,256)


In [5]:
def crop_char_image(image, threshold=5./255.):
    assert image.ndim == 2
    is_white = image < threshold
    is_black_vertical = np.sum(is_white, axis=0) > 0
    is_black_horizontal = np.sum(is_white, axis=1) > 0
    left = np.argmax(is_black_horizontal)
    right = np.argmax(is_black_horizontal[::-1])
    top = np.argmax(is_black_vertical)
    bottom = np.argmax(is_black_vertical[::-1])
    height, width = image.shape
    cropped_image = image[left:height - right, top:width - bottom]
    return cropped_image

In [6]:

# dataloader that combines the images and labels into one convenient class
# also uses the make_square function to reshape the images
class BengaliDataLoader(Dataset):
    def __init__(self,images,labels=None, transform=None, indices=None):
        self.images = images.drop('image_id', axis='columns')
        self.images = self.images
        self.labels = labels
        if indices is None:
            indices = np.arange(len(self.images))
        self.indices = indices
        self.train = labels is not None
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self,idx):
#         idx = self.indices[idx]
        img = np.zeros((256, 256, 3))
        tmp = self.images.iloc[idx].to_numpy().reshape(137,236)
#         tmp = (255 - tmp).astype(np.float32) / 255.
        tmp = tmp/255. #tmp.astype(np.float32)/255.

        tmp = crop_char_image(tmp,threshold = 250./255.)
        tmp = res(image=tmp)['image']
        img[..., 0] = tmp
        img[..., 1] = img[..., 0]
        img[..., 2] = img[..., 0]


        x = torch.from_numpy(img)
        if self.transform:
            x = x.reshape(3,256,256)
            x = self.transform(x)

#             x = x.reshape(256,256,3)
            
        if self.train:
            y = self.labels[idx]
            y = torch.from_numpy(y)
            return x,y
        else:
            return x

In [7]:
# finetuning the network
class ResNet50(torch.nn.Module):
    def __init__(self,):
        super(ResNet50, self).__init__()

        self.resnet50 = resnet50(pretrained=False,progress=False)
#         for param in self.resnet50.parameters():
#             param.requires_grad = False

        self.learned_features =  nn.Sequential(*list(self.resnet50.children())[:-1])
        
        #self.in_features = self.resnet50.fc.in_features

#         self.fc0 = nn.Linear(32768,256)
        # grapheme_root
        self.sm = nn.Softmax(dim=-1)
        self.fc1 = nn.Linear(2048,168)
        # vowel_diacritic 
        self.fc2 = nn.Linear(2048,11)
        # consonant_diacritic
        self.fc3 = nn.Linear(2048,7)

    def forward(self, x):
#         x = self.resnet50(x)
        x = self.learned_features(x)
#         print(x.shape)
#         print(torch.flatten(x,1).shape)
        x = x.view(x.size(0), -1)
#         print(x.shape)
#         x = self.fc0(x)
        x1 = self.sm(x)
        x1 = self.fc1(x)
        x2 = self.sm(x)
        x2 = self.fc2(x)
        x3 = self.sm(x)
        x3 = self.fc3(x)
        return x1,x2,x3

In [8]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
composed = transforms.Compose([normalize])

In [9]:
# load model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model = SeResNext50().to(device)

model_ft = ResNet50().to(device)
state_dict = torch.load('/kaggle/input/resnet5091acc/acc91_resnet50_saved_weights_1585840262.pth',map_location=torch.device('cpu'))

# for key, value in state_dict.items():
#     if 'resnet50' in key:
#         new_name = 'se_resnext50_32x4d' + key[8:]
#         print(key)
#         print(new_name)
#         print('00000000000000')
#         state_dict[new_name] = state_dict.pop(key)
# # print(state_dict)
model_ft.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
# make predictions
model_ft.eval()
test_data = ['test_image_data_0.parquet','test_image_data_1.parquet','test_image_data_2.parquet','test_image_data_3.parquet']
predictions = []
for fname in test_data:
    test_dataset = BengaliDataLoader(pd.read_parquet(f'/kaggle/input/bengaliai-cv19/{fname}'), transform=composed)
    test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False)
    with torch.no_grad():
        for idx, (inputs) in tqdm(enumerate(test_dataloader),total=len(test_dataloader)):
            inputs = inputs.float().to(device)
            outputs = model_ft(inputs)

            _, preds_c1 = torch.max(outputs[0], 1)
            _, preds_c2 = torch.max(outputs[1], 1)
            _, preds_c3 = torch.max(outputs[2], 1)
            
            predictions.append(preds_c3.cpu().numpy()) # first add consonant
            predictions.append(preds_c1.cpu().numpy()) # then grapheme
            predictions.append(preds_c2.cpu().numpy()) # then vowel

# save results
submission = pd.read_csv(f'/kaggle/input/bengaliai-cv19/sample_submission.csv')
submission.target = np.hstack(predictions)
submission.to_csv('submission.csv',index=False)

100%|██████████| 3/3 [00:00<00:00,  6.38it/s]
